# Plot harmonic measure on a GW-tree as apparent after some finite number of generations

In [45]:
import networkx as nx
import markovmixing as mkm

import numpy
import matplotlib.pyplot as plt

execfile('graph_util.py')

## Choose one of these offspring distributions

In [94]:
offspring = lambda: numpy.random.poisson(1.5)
generation_measure = 8
generation_hit = 13

In [103]:
offspring = lambda: numpy.random.choice([1,2])
generation_measure = 8
generation_hit = 15

In [67]:
offspring = lambda: numpy.random.choice([1,2,3])

## Generate a GW-tree with the chosen offspring

In [104]:
numpy.random.seed(0)

G = grow_gw_tree(offspring, n_gen = generation_hit, directed = True)

while nx.number_of_nodes(G) <= 100: 
	G = grow_gw_tree(offspring, n_gen = generation_hit, directed = True)
    
print nx.number_of_nodes(G)

1578


In [83]:
# Show the tree
show_tree(G)

## Setup for the simulation

In [106]:
# The tree up to the generation where we want to determine the harmonic measure
G_measure = G.copy()

for node in G_measure.nodes():
	if G_measure.node[node]['generation']>generation_measure:
			G_measure.remove_node(node)
            
# the nodes of generation measure and hit
generation = nx.get_node_attributes(G, 'generation')

generation_measure_nodes = [key for key in generation if generation[key] == generation_measure]
generation_hit_nodes = [key for key in generation if generation[key] == generation_hit]

# dictionary mapping nodes of generation hit to their ancestors in generation measure
ancestor_dict = {}

for nh in generation_hit_nodes:
    a = nh
    
    while generation[a] != generation_measure:
        a = G.predecessors(a)[0] 
        
    ancestor_dict[nh] = a
        
print generation_measure_nodes
print len(generation_hit_nodes)

[8, 89, 162, 241, 282, 298, 329, 384, 420, 471, 546, 584, 639, 709, 788, 858, 917, 988, 1004, 1048, 1106, 1167, 1183, 1208, 1238, 1277, 1301, 1349, 1394, 1437, 1453, 1470, 1496, 1513, 1543]
527


In [107]:
# Show the tree up to generation measure
show_tree(G_measure)

## Monte-Carlo simulation of SRW on the tree

In [110]:
# monte-carlo simulation of the harmonic measure 
mc = mkm.nx_graph_srw(G.to_undirected())
nhits = {}

for n in generation_measure_nodes:
    nhits[n] = 0

for i in xrange(1000):
    if i % 50 == 0:
        print i
    
    # start a SRW at the root
    path = [1]
    hit = -1

    while hit == -1:
        # let the SRW walk 99 steps
        path = mc.sample_path(path[-1],100)

        # did we hit generation hit?
        for x in path:
            if generation[x] == generation_hit:
                hit = x
                break
    
    # log the hit
    nhits[ancestor_dict[hit]] = nhits[ancestor_dict[hit]]+1

print nhits
print sum(nhits.values())

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
{384: 12, 1543: 52, 8: 28, 1167: 21, 788: 56, 917: 26, 1048: 15, 282: 24, 1437: 18, 1183: 20, 162: 33, 420: 27, 298: 46, 1453: 13, 1301: 17, 1208: 28, 1470: 40, 709: 60, 1349: 28, 584: 32, 329: 54, 546: 8, 1106: 22, 1238: 36, 471: 25, 1496: 50, 89: 25, 858: 14, 988: 6, 1513: 51, 1004: 10, 241: 14, 1394: 21, 1277: 13, 639: 55}
1000


### Plot the distribution of hits

In [112]:
x = nhits.keys()

y = []
for key in x:
    y.append(nhits[key]/100.0)

fig = plt.figure()

plt.bar(numpy.arange(len(x)), y, align='edge', width=1.0)

plt.title("Monte-Carlo estimate of harmonic measure")
#plt.xlabel("Markov chain state space")
plt.ylabel("Probabiliy")
#plt.tick_params(axis='x', which='both', bottom='off', top='off', labelbottom='off')

plt.xlim(0, len(x))
#plt.ylim(0, 1)
plt.show()

